In [ ]:
#| default_exp reading
%load_ext autoreload
%autoreload 2

import sys
from fastcore.xtras import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Leitura
> Este módulo concentra funções para a leitura dos diversos arquivos que compõe a base de dados

* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão Comunitária
* `MOSAICO` - Demais serviços de Radiodifusão, e.g. TV, RTV, FM, AM, etc.
* `AERONAUTICA` - Consolidação de diversas bases de dados públicas da aeronáutica 

In [ ]:
#| export
import os
from typing import Union
from pathlib import Path

import pandas as pd
import pyodbc
from pymongo import MongoClient
from dotenv import load_dotenv, find_dotenv

from extracao.updates import (
    update_srd,
    update_stel,
    update_radcom,
    update_base,
    update_telecom,
    update_aero,
)

from extracao.icao import get_icao
from extracao.aisgeo import get_aisg
from extracao.aisweb import get_aisw
from extracao.redemet import get_redemet
from extracao.format import _read_df

load_dotenv(find_dotenv())

True

## MOSAICO
> O mosaico atualmente é composto por 2 bases complementares originárias de um banco `MongoDB`: 

-  `Estações`
- `Plano Básico`

In [ ]:
#| export
def read_srd(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: MongoClient = None,  # Objeto de Conexão com o banco MongoDB, atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados do mosaico
    """Lê o banco de dados salvo localmente do MOSAICO e opcionalmente o atualiza."""
    return update_srd(conn, folder) if conn else _read_df(folder, "srd")


In [ ]:
folder = Path.cwd().parent / 'dados'

In [ ]:
from extracao.constants import MIN_LAT, MAX_LAT, MIN_LONG, MAX_LONG
# from pandas_profiling import ProfileReport

In [ ]:
mosaico = read_srd(folder)

In [ ]:
mosaico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30849 entries, 0 to 30848
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Frequência            30849 non-null  string
 1   Entidade              30845 non-null  string
 2   Fistel                30849 non-null  string
 3   Número_Estação        27348 non-null  string
 4   Município             30849 non-null  string
 5   Código_Município      30849 non-null  string
 6   UF                    30849 non-null  string
 7   Latitude              15043 non-null  string
 8   Longitude             15043 non-null  string
 9   Classe                30840 non-null  string
 10  Num_Serviço           30849 non-null  string
 11  Classe_Emissão        0 non-null      string
 12  Largura_Emissão(kHz)  30849 non-null  string
 13  Validade_RF           28727 non-null  string
 14  Status                30849 non-null  string
 15  Fonte                 30849 non-null

In [ ]:
for c in ['Latitude', 'Longitude']:
    mosaico[c] = mosaico[c].astype('float')

In [ ]:
# mos_bad_coords = mosaico[~(mosaico.Latitude.between(MIN_LAT, MAX_LAT) | mosaico.Longitude.between(MIN_LONG, MAX_LONG))]
# mos_bad_coords

In [ ]:
# print(f"A base de radiodifusão do Mosaico possui {len(mos_bad_coords)} registros de coordenadas inválidas do total de {len(mosaico)}")

In [ ]:
# mosaico['Frequência'] = mosaico['Frequência'].astype('category')
# mosaico_profile = ProfileReport(mosaico, config_file='report_config.yaml', title='MOSAICO')
# mosaico_profile.to_notebook_iframe()

## LICENCIAMENTO
> A Base licenciamento é composta de serviços privados de telecomunicações, tanto antigos migrados do STEL como novos para um novo banco de dados `MongoDB`

In [ ]:
#| export
def read_telecom(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: MongoClient = None,  # Objeto de Conexão com o banco MongoDB, atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados do mosaico
    """Lê o banco de dados salvo localmente do LICENCIAMENTO e opcionalmente o atualiza."""
    return update_telecom(conn, folder) if conn else _read_df(folder, "telecom")


In [ ]:
#|eval: false
#|code_fold: true
telecom = read_telecom(folder)

In [ ]:
#|eval: false
for c in ['Latitude', 'Longitude']:
    telecom[c] = telecom[c].astype('float')

In [ ]:
#| eval: false
# telecom[~(telecom.Latitude.between(MIN_LAT, MAX_LAT) | telecom.Longitude.between(MIN_LONG, MAX_LONG))]

In [ ]:
#| eval: false
# telecom['Frequência'] = telecom['Frequência'].astype('category')
# telecom_profile = ProfileReport(telecom, config_file='report_config.yaml', title='Telecomunicações')
# telecom_profile.to_notebook_iframe()

##  RADCOM

In [ ]:
#| export
def read_radcom(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: pyodbc.Connection = None,  # Objeto de conexão de banco, atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados de RADCOM
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO05 caso `update = True` ou não exista o arquivo local"""
    return update_radcom(conn, folder) if conn else _read_df(folder, "radcom")


In [ ]:
#|eval: false
#|code_fold: true
radcom = read_radcom(folder)
# radcom['Frequência'] = radcom['Frequência'].astype('category')
# radcom_profile = ProfileReport(radcom, config_file='report_config.yaml', title='RADCOM')
# radcom_profile.to_notebook_iframe()

## STEL

In [ ]:
#| export
def read_stel(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: pyodbc.Connection = None,  # Objeto de conexão de banco. Atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados do STEL
    """Lê o banco de dados salvo localmente do STEL.
     Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO05
    caso `update = True` ou não exista o arquivo local"""
    return update_stel(conn, folder) if conn else _read_df(folder, "stel")


In [ ]:
#|eval: false
#|code_fold: true
stel = read_stel(folder)
# stel['Frequência'] = stel['Frequência'].astype('category')
# stel_profile = ProfileReport(stel, config_file='report_config.yaml', title='STEL')
# stel_profile.to_notebook_iframe()

In [ ]:
from extracao.constants import SQL_VALIDA_COORD

In [ ]:
def valida_coords(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    row: pd.Series,  # Linha de um DataFrame
) -> tuple:  # DataFrame com dados do município
    """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativo"""
    
    sql = SQL_VALIDA_COORD.format(
        row.Longitude, row.Latitude, row.Código_Município
    )
    crsr = conn.cursor()
    crsr.execute(sql)
    result = crsr.fetchone()
    if result is None:
        return (row.Código_Município, row.Longitude, row.Latitude, -1)
    elif result.COORD_VALIDA == 1:
        return result
    else:
        return (
            result.NO_MUNICIPIO,
            result.NU_LONGITUDE,
            result.NU_LATITUDE,
            result.COORD_VALIDA,
        )

## Bases Externas da Aeronáutica

In [ ]:
#| export
def read_icao(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do ICAO
    """Lê a base de dados do Frequency Finder e Canalização VOR/ILS/DME"""
    return get_icao if update else _read_df(folder, "icao")


In [ ]:
#| export
def read_aisw(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do AISWEB
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    return get_aisw() if update else _read_df(folder, "aisw")


In [ ]:
#| export
def read_aisg(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do GEOAISWEB
    """Fontes da informação: GEOAISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    return get_aisg() if update else _read_df(folder, "aisg")


In [ ]:
#| export
def read_redemet(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do AISWEB
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    return get_redemet() if update else _read_df(folder, "redemet")


In [ ]:
#| export
def read_aero(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados mesclados das 3 bases da Aeronáutica anteriores
    """Lê os arquivos de dados da aeronáutica e retorna os registros comuns e únicos"""
    return update_aero(folder) if update else _read_df(folder, "aero")


In [ ]:
#| eval: false
aero = read_aero(folder)
# aero['Frequency'] = aero['Frequency'].astype('category')
# aero_profile = ProfileReport(aero, config_file='report_config.yaml', title='AERONAUTICA')
# aero_profile.to_notebook_iframe()

## Base Consolidada

In [ ]:
#| export
def read_base(
    folder: Union[str, Path],
    conn: pyodbc.Connection = None,  # Objeto de conexão do banco SQL Server
    clientMongoDB: MongoClient = None,  # Objeto de conexão do banco MongoDB
) -> pd.DataFrame:
    """Lê a base de dados e opcionalmente a atualiza antes da leitura casos as conexões de banco sejam válidas"""
    return (
        update_base(conn, clientMongoDB, folder)
        if all([conn, clientMongoDB])
        else _read_df(folder, "base")
    )

In [ ]:
#|eval: false
#|code_fold: true
base = read_base(folder)
# base['Frequência'] = base['Frequência'].astype('category')
# base_profile = ProfileReport(base, config_file='report_config.yaml', title='Base Consolidada')
# base_profile.to_notebook_iframe()